In [1]:
import time

import tensorflow as tf
import numpy as np

import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import custom_vgg19

In [2]:
def load_image(path):
    img = skimage.io.imread(path)
    img = img / 255.0
    # add another dimension to make it a batch , bacause our vgg19 def takes a batch
    img = img.reshape((1,)+img.shape)
    return img

In [3]:
path = './vd.jpg'
img = skimage.io.imread(path)
img = img / 255.0
img = np.expand_dims(img,0)
print(img.shape)
# show image
# skimage.io.imshow(img)
# plt.show()

(1, 337, 236, 3)


In [4]:
sess=tf.Session()
img_pl = tf.placeholder(tf.float32)
vgg19factory = custom_vgg19.Vgg19Factory()
vgg19 = vgg19factory.build(img_pl)

/work/04006/zhuode93/maverick/dlproj2/trainable-neural-style/tensorflow_vgg/vgg19.npy
npy file loaded


In [5]:
print(img.shape)
conv21feat = sess.run(vgg19.conv2_1, feed_dict={img_pl:img})
print(conv21feat.shape)

(1, 337, 236, 3)
(1, 169, 118, 128)


In [6]:
def gram_matrix(feat_map):
    assert isinstance(feat_map, tf.Tensor)
    shape = tf.shape(feat_map)
    h, w, ch = tf.cast(shape[1], tf.float32), tf.cast(shape[2], tf.float32), shape[3]
    size = h*w
    F = tf.reshape(feat_map, [-1, ch])
    
    # TODO: if m<n, compute feat_map*feat_map, else compute feat_map'*feat_map 
    gram = tf.matmul(F, F, True) / (size**2)
    return gram
    

In [11]:
img_gen = tf.Variable(tf.truncated_normal(img.shape, mean=0.5, stddev=0.25))
vgg19_for_img_gen = vgg19factory.build(img_gen)

G = gram_matrix(vgg19.conv3_1)

In [38]:
# compute gram_matrix for the style image
style_gram = tf.Variable(G,validate_shape=False,trainable=False)

styleimg_content = tf.Variable(vgg19.conv2_1,validate_shape=False, trainable=False)

sess.run(tf.initialize_all_variables(), feed_dict={img_pl:img})

gen_gram = gram_matrix(vgg19_for_img_gen.conv3_1)
shape = tf.cast(tf.shape(gen_gram), tf.float32)
style_loss = tf.nn.l2_loss(gen_gram - style_gram) # / shape[0] / shape[1] # uncomment this will cause img_gen
                        # stay being white-noise. see what is the magnitude of the cost of stylenet?
[style_loss_np, shapenp] =sess.run([style_loss, shape])
print('initial style loss = %f' % style_loss_np)

shape = tf.cast(tf.shape(vgg19_for_img_gen.conv2_1), tf.float32)
nElement = tf.reduce_prod(shape)
content_loss = tf.nn.l2_loss(styleimg_content - vgg19_for_img_gen.conv2_1) / nElement 
[content_loss_np, shapenp] =sess.run([content_loss, shape])
print('initial content loss = %f' % content_loss_np)


initial style loss = 91256.062500
initial content loss = 12901.880859


In [39]:
optimizer = tf.train.RMSPropOptimizer(0.005)
total_loss = style_loss + content_loss
train_op = optimizer.minimize(total_loss)
# start optimization
iter = 0
MAX_ITER = 1000
sess.run(tf.initialize_all_variables(), feed_dict={img_pl:img})
while iter < MAX_ITER:
    sess.run(train_op)
    iter += 1
print(sess.run(total_loss))

220.353


In [44]:
img_gen_np = np.squeeze(sess.run(img_gen), 0)
minv,maxv = np.min(img_gen_np), np.max(img_gen_np)
print(minv,maxv) # TODO: I don't know why minv<0, maxv>1,
     # and this make imshow shows a more "grey" image compared to the original one
img_gen_np =  (img_gen_np-minv)/(maxv-minv)
skimage.io.imshow(img_gen_np)
plt.show()

(-0.62544727, 1.5940531)
(1.0, 0.0)


In [41]:
print img.shape
print(  np.min(img[0,:,:,:]), np.max(img[0,:,:,:]))
skimage.io.imshow(img[0,:,:,:])
plt.show()

(1, 337, 236, 3)
(0.0, 1.0)
